In [1]:
from kafka import KafkaConsumer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import clear_output
import json

topic = 'streams-input'
window_size = 100
bootstrap_server = 'localhost:9092'

In [2]:
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=[bootstrap_server],
    value_deserializer=lambda m: json.loads(m.decode('ascii')),
    auto_offset_reset='earliest'  # Adjust this as needed
)

In [3]:
# Initialize an empty list to store message values
messages = np.array([])

fig, axs = plt.subplots(2)
line, = axs[0].plot([], [])

axs[0].set_xticklabels([])
axs[0].set_xticks([])
axs[0].set_ylabel('Message Value')
axs[0].set_title(f'Streamed Values for Topic: {topic}')

sns.histplot(messages, ax=axs[1], color='blue')
axs[1].set_xlabel('Values')
axs[1].set_ylabel('Frequency')
axs[1].set_title('Value Distribution')

while True:
    records = consumer.poll(timeout_ms=1000)
    for topic_data, consumer_records in records.items():
        for consumer_record in consumer_records:
            messages = np.append(messages, consumer_record.value['value'])
    line.set_data(range(len(messages[-window_size:])), messages[-window_size:])
    
    axs[1].cla()
    sns.histplot(messages, ax=axs[1], color='blue')
    axs[1].set_xlabel('Values')
    axs[1].set_ylabel('Frequency')
    axs[1].set_title('Value Distribution')
    axs[1].relim()
    axs[1].autoscale_view()

    axs[0].relim()
    axs[0].autoscale_view()
    display(fig)
    
    clear_output(wait = True)
    plt.pause(1)
    

KeyboardInterrupt: 